# HyperCLOVA X 캐릭터챗봇 튜닝용 데이터셋 질문생성기
📅 2024-07-11    
🧑🏻‍💻 외계공룡
    
> *colab(https://colab.research.google.com/)에 접속해서 스크립트(`hcx_character_question_generator.ipynb`)를 불러온 후 사용하는것을 권장드립니다.*

In [ ]:
!python --version # 파이썬 버전확인

Python 3.10.12


# 🚀 실행 순서
1. API 정보 입력
2. 프롬프트 작성
3. 코드 실행

# 1. API 정보 입력
❗하단에 CLOVA Studio 테스트 앱 API 정보 입력❗

In [1]:
X_NCP_CLOVASTUDIO_API_KEY='nv-e58bfc47db0544f59ea592a69d4a94c35PUB'
X_NCP_APIGW_API_KEY='X-NCP-APIGW-API-KEY'
X_NCP_CLOVASTUDIO_REQUEST_ID='40c9748815c84867ae1496148e404b1d'

# 2. 프롬프트 입력
🪄 캐릭터 질문생성 프롬프트 TIP
> 1. 원하는 캐릭터 선정 (ex. 귀멸의칼날 '카마도 탄지로')
> 2. 역할 부여 (Role Assistant)
> 3. 형식 지정 (Formatted Prompt)
>    - 정보 : 캐릭터 정보 입력 (나무위키 링크 크롤링 가능)


In [2]:
prompt = '''
당신은 일본 애니메이션 귀멸의 칼날을 시청한 상태입니다.
상대방이 탄지로라고 가정하고 질문 생성 방법을 바탕으로 궁금한 점들을 100개 이상 질문하세요.

### 질문 생성 방법 ###
- 대화체로 생성한다.
- 질문은 전부 반말로 생성한다.
- 줄거리와 관련된 질문들을 많이 생성한다.
- 질문은 다음과 같은 형식으로 출력한다. [\"첫 번째 질문\", \"두 번째 질문\", ...]
'''

# 3. 코드 실행
`execute_count` 수 만큼 반복문 실행됨 (💸 과금 주의 )

In [4]:
execute_count=40

In [5]:
import ast
import csv
import os
import random
import requests
import time
import json

class QuestionGenerator:
    host='https://clovastudio.stream.ntruss.com'
    def __init__(self):
        pass

    def execute(self):
        temperature = round(random.uniform(0.2, 0.95), 2)
        print("temperature : {}".format(temperature))

        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': X_NCP_CLOVASTUDIO_API_KEY,
            # 'X-NCP-APIGW-API-KEY': X_NCP_APIGW_API_KEY,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': X_NCP_CLOVASTUDIO_REQUEST_ID,
            'Content-Type': 'application/json',
        }

        request_body = {
            'messages': [
                {"role":"system","content":prompt},
                {"role":"user","content":"만약 생성 길이에 제한이 걸린 다면 제한을 무시하세요."}
            ],
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 2000,
            'temperature': temperature,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        response = requests.post(self.host + '/testapp/v1/chat-completions/HCX-DASH-001',headers=headers, json=request_body, stream=False, timeout=600)
        json_data = json.loads(response.text)
        print("token : {}".format(json_data['result']['outputLength']))
        return json_data['result']['message']['content']

if __name__ == '__main__':
    file_name = 'datasets.csv'
    index = 0
    completion_generator = QuestionGenerator()

    for i in range(execute_count):
        if not os.path.exists(file_name):
            with open(file_name, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['C_ID', 'T_ID', 'Text', 'Completion'])

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])
        else:
            with open(file_name, mode='r+', newline='', encoding='utf-8') as file:
                reader = csv.reader(file)
                rows = list(reader)
                index = int(rows[-1][0]) + 1 if len(rows) > 1 else 0
                file.seek(0, os.SEEK_END)
                writer = csv.writer(file)

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])
        time.sleep(15)


temperature : 0.34


KeyError: 'result'